<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Notebook: Utility to confirm connectivity between Db2 and OpenScale

Target Version: Cloud pak for Data **v4.8.X**

The user needs to provide the necessary inputs (where marked) to be able to proceed. 

**Note**: This notebook is designed to test connection to Db2

**Contents:**
1. [Package Installation](#Package-installation)
2. [User Inputs](#User-Inputs)
3. [Setting up Services](#Setting-up-Services)
4. [Collect resources to be migrated](#Collect-resources-to-be-migrated)

## Package installation

In [ ]:
import warnings
warnings.filterwarnings('ignore')
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

Action: restart the kernel if you executed above cell!

## User Inputs

The following inputs are required:

1. **IBM_CPD_ENDPOINT:** The URL representing the IBM Cloud Pak for Data service endpoint.
2. **IBM_CPD_USERNAME:** IBM Cloud Pak for Data username used to obtain a bearer token.
3. **IBM_CPD_PASSWORD:** IBM Cloud Pak for Data password used to obtain a bearer token.
4. **CPD_API_KEY:** IBM Cloud Pak for Data API Key used to obtain a bearer token.

## User Inputs

The following inputs are required:

1. **IBM_CPD_ENDPOINT:** The URL representing the IBM Cloud Pak for Data service endpoint.
2. **IBM_CPD_USERNAME:** IBM Cloud Pak for Data username used to obtain a bearer token.
3. **IBM_CPD_PASSWORD:** IBM Cloud Pak for Data password used to obtain a bearer token.
4. **CPD_API_KEY:** IBM Cloud Pak for Data API Key used to obtain a bearer token.

In [2]:
# IBM Cloud Pak for Data credentials
IBM_CPD_ENDPOINT = "<The URL representing the IBM Cloud Pak for Data service endpoint.>"
IBM_CPD_USERNAME = "<IBM Cloud Pak for Data username used to obtain a bearer token.>"
IBM_CPD_PASSWORD = "<IBM Cloud Pak for Data password used to obtain a bearer token.>"
SERVICE_INSTANCE_ID = "<SERVICE_INSTANCE_ID>" #Default is 00000000-0000-0000-0000-000000000000
CPD_API_KEY ="<API KEY>" 

## Setting up Services

In [ ]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient

service_credentials = {
                "url": IBM_CPD_ENDPOINT,
                "username": IBM_CPD_USERNAME,
                "password": IBM_CPD_PASSWORD,
                # "apikey": CPD_API_KEY
                }

authenticator = CloudPakForDataAuthenticator(
        url=service_credentials['url'],
        username=service_credentials['username'],
        password=service_credentials['password'],
        # apikey=service_credentials['apikey'],
        disable_ssl_verification=True
)

client = APIClient(
    service_url=service_credentials['url'],
    service_instance_id=SERVICE_INSTANCE_ID,
    authenticator=authenticator
)

print(client.version)

## Listing schemas in the specified database

In [ ]:
body = {
    "database_type": "db2",
    "credentials": {
        "hostname": "<HOST for db2>",
        "username": "<Username for the database>",
        "password": "<Passowrd for the databasse>",
        "db": "<DATABASE name to check>",
        "port": 50000,
        "ssl": False,
        # if you need to set it true, depending on your env
        "certificate_base64": "base64 encoded ssl certificate if you need",
        "additional_properties": {
            "securityMechanism": 9,
            # "encryptionAlgorithm": 2
        }
        # this field is not necessary, but you can append more parameters 
    }
}
# You can adjust parameters in `credentials` for your need


pr = client.prepare_request("POST", "/v2/database_schemas", data=body)
result = client.send(pr).get_result()
result

## result should list schemas in database where user is authorized to see

{'database_schemas': [{'compatible': True, 'name': 'DB2INST1'},
  {'compatible': True, 'name': 'NULLID'},
  {'compatible': True, 'name': 'SQLJ'},
  {'compatible': True, 'name': 'SYSCAT'},
  {'compatible': True, 'name': 'SYSFUN'},
  {'compatible': True, 'name': 'SYSIBM'},
  {'compatible': True, 'name': 'SYSIBMADM'},
  {'compatible': True, 'name': 'SYSIBMINTERNAL'},
  {'compatible': True, 'name': 'SYSIBMTS'},
  {'compatible': True, 'name': 'SYSPROC'},
  {'compatible': True, 'name': 'SYSPUBLIC'},
  {'compatible': True, 'name': 'SYSSTAT'},
  {'compatible': True, 'name': 'SYSTOOLS'}]}

## Getting database configuration

In [ ]:
dataMart = client.data_marts.list(limit = 1)
secretId = dataMart.result.data_marts[0].entity.database_configuration.credentials.secret_id
print(secretId)

SECRET_URL = "/v2/secrets/{}".format(secretId)
pr = client.prepare_request("GET", SECRET_URL)
result = client.send(pr).get_result()
result

### Add parameters to database configuration

In [ ]:
import json
patch_document = [
    {
        "op": "add",
        "path": "/credentials/additional_properties",
        "value": {
            "securityMechanism": 9,
            "encryptionAlgorithm": 2
        }
    }
]

data = json.dumps(patch_document)
print(data)
pr = client.prepare_request(method="PATCH", url=SECRET_URL,  data=data)
result = client.send(pr).get_result()
result

## Revert changes in database configuration

In [ ]:
import json
patch_document = [
    {
        "op": "remove",
        "path": "/credentials/additional_properties",
    }
]

data = json.dumps(patch_document)
print(data)
pr = client.prepare_request(method="PATCH", url=SECRET_URL,  data=data)
result = client.send(pr).get_result()
result